# LSTM poem generation for Shakespeare's sonnets

In [78]:
import numpy as np
import itertools

Reference: https://machinelearningmastery.com/text-generation-lstm-recurrent-neural-networks-python-keras/

# Data preprocessing for LSTM

In [79]:
# Function to get Shakespeare's poems from file
def getPoems():
    with open("./data/shakespeare.txt", "r") as f:
        data = f.read().lower()
    # Split by poems
    poems = data.split("\n\n\n")
    # Remove 1st line of each poem
    poems = [poem[20:] for poem in poems]
    return poems

# Get character to integer dictionary for one hot encoding
def getChardict(poems):
    # merge all poems and get list of characters
    data = "".join(poems)
    # Get dictionary of characters for one hot encoding
    chars = sorted(list(set(data)))
    charint = dict((c, i) for i, c in enumerate(chars))
    intchar = dict((i, c) for i, c in enumerate(chars))
    return charint,intchar

# Integer encode the poems
def getIntPoems(charint,poems):
    out = []
    for poem in poems:
        out.append([charint[char] for char in poem])
    return np.array(out)

# Get array of poems
poems = getPoems()
# Get integer encoding dictionary
charint,intchar = getChardict(poems)
# Get Integer encoded poem array
IntPoems = getIntPoems(charint,poems)

# Generate training data

In [197]:
# Generate X and Y training sets from each poem
def getCharacters(poem,n=40,skip=10):
    Xtrain = [poem[i:i+n] for i in range(0,len(poem)-n,skip)]
    Ytrain = [poem[i+n] for i in range(0,len(poem)-n,skip)]
    return Xtrain,Ytrain

# Generate training data 
Ntime = 40
skip = 5
Xtrain = []
Ytrain = []
for poem in IntPoems:
    Xt,Yt = getCharacters(poem,n,skip)
    Xtrain.append(Xt)
    Ytrain.append(Yt)

Ytrain = np.concatenate(Ytrain)
Xtrain = np.concatenate(Xtrain)

# One hot encode the training vectors
import keras
Yt = keras.utils.np_utils.to_categorical(Ytrain)
Xt = keras.utils.np_utils.to_categorical(Xtrain)

print(Xt.shape)
print(Yt.shape)

(17735, 40, 48)
(17735, 48)


# LSTM Model

In [207]:
from keras.layers import LSTM, Dense, Activation, BatchNormalization, Dropout
from keras.models import Sequential
from keras.callbacks import ModelCheckpoint

Nchars = len(charint)
Ntime = 40

model = Sequential()
model.add(LSTM(200, input_shape=(Xt.shape[1],Xt.shape[2])))
model.add(Dropout(0.0))
model.add(Dense(Nchars))
model.add(Activation('softmax'))
model.summary()

model.compile(loss='categorical_crossentropy', optimizer='adam')

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_21 (LSTM)               (None, 200)               199200    
_________________________________________________________________
dropout_15 (Dropout)         (None, 200)               0         
_________________________________________________________________
dense_17 (Dense)             (None, 48)                9648      
_________________________________________________________________
activation_15 (Activation)   (None, 48)                0         
Total params: 208,848
Trainable params: 208,848
Non-trainable params: 0
_________________________________________________________________


In [ ]:
# define the checkpoint
fname="./data/Data_LSTM-{epoch:02d}-{loss:.4f}.hdf5"
checkpoint = ModelCheckpoint(fname, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]
model.fit(Xt, Yt, epochs=20, batch_size=128, callbacks=callbacks_list)

Epoch 1/20
17664/17735 [============================>.] - ETA: 0s - loss: 3.0411
Epoch 00001: loss improved from inf to 3.04025, saving model to ./data/Data_LSTM-01-3.0402.hdf5
17735/17735 [==============================] - 30s 2ms/step - loss: 3.0402
Epoch 2/20
17664/17735 [============================>.] - ETA: 0s - loss: 2.7785
Epoch 00002: loss improved from 3.04025 to 2.77791, saving model to ./data/Data_LSTM-02-2.7779.hdf5
17735/17735 [==============================] - 28s 2ms/step - loss: 2.7779
Epoch 3/20
17664/17735 [============================>.] - ETA: 0s - loss: 2.4644
Epoch 00003: loss improved from 2.77791 to 2.46438, saving model to ./data/Data_LSTM-03-2.4644.hdf5
17735/17735 [==============================] - 32s 2ms/step - loss: 2.4644
Epoch 4/20
17664/17735 [============================>.] - ETA: 0s - loss: 2.3339
Epoch 00004: loss improved from 2.46438 to 2.33355, saving model to ./data/Data_LSTM-04-2.3335.hdf5
17735/17735 [==============================] - 28s 2ms/

# Generate poems from training set

In [193]:
def CharToInt(charint,text):
    return np.array([charint[char] for char in text])
    
def IntToChar(intchar,text):
    return "".join([intchar[char] for char in text])

# helper function to sample an index from a probability array
def sample(a, temperature=1.0):
    a = np.log(a) / temperature
    a = np.exp(a)
    a = a/np.sum(a)*.99
    return np.argmax(np.random.multinomial(1, a, 1))

def generatePoem(model,seed,temp=1.0):
    print('Seed = ',seed)
    IntSeed = CharToInt(charint,seed)
    print('Integer Encoded Seed=',IntSeed)

    IntOut = IntSeed
    temp = 1.0
    lines = 13
    # generate characters
    for i in range(1000):
        X = IntOut[i:i+Ntime]
        OneHot_X = keras.utils.np_utils.to_categorical([X],num_classes=len(charint))
        Ypred = model.predict(OneHot_X)
        idx = sample(Ypred[0],temp)
        IntOut = np.concatenate((IntOut,[idx]))
        # Count number of poem lines generated
        if idx==0:
            lines-=1
        if lines==0:
            break
    return IntToChar(IntOut)

Seed =  shall i compare thee to a summer's day?

Integer Encoded Seed= [40 29 22 33 33  1 30  1 24 36 34 37 22 39 26  1 41 29 26 26  1 41 36  1
 22  1 40 42 34 34 26 39  3 40  1 25 22 46 21  0]


In [ ]:
seed = "shall i compare thee to a summer's day?\n"
temp = [1.5,0.75,0.25]
for i in temp:
    print('Generated Poem at temp = ',i,':')
    print(generatePoem(model,seed,temp=i))

# LSTM model 2

In [ ]:
# 2 layer LSTM
Nchars = len(charint)
Ntime = 40

model2 = Sequential()
model2.add(LSTM(200, input_shape=(Xt.shape[1],Xt.shape[2])))
model2.add(Dropout(0.0))
model2.add(LSTM(200, input_shape=(Xt.shape[1],Xt.shape[2])))
model2.add(Dropout(0.0))
model2.add(Dense(Nchars))
model2.add(Activation('softmax'))
model2.summary()

model2.compile(loss='categorical_crossentropy', optimizer='adam')

In [ ]:
# define the checkpoint
fname="./data/Data_LSTM-{epoch:02d}-{loss:.4f}.hdf5"
checkpoint = ModelCheckpoint(fname, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]
model2.fit(Xt, Yt, epochs=20, batch_size=128, callbacks=callbacks_list)

In [ ]:
seed = "shall i compare thee to a summer's day?\n"
temp = [1.5,0.75,0.25]
for i in temp:
    print('Generated Poem at temp = ',i,':')
    print(generatePoem(model,seed,temp=i))